In [83]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [2]:
ESPN_PP_PREFIX_LINK = 'http://games.espn.com/ffl/tools/projections?startIndex=' #after '=' is a value perfectly divisible by 40, i.e. 0, 40, 80...

In [92]:
def createDataframe(prefix,mod=40,max_=560):
    '''
    Uses a list of integers that correspond to ranks that are located in the 2018
    projections list. This list will then be used to create a dataframe with the projections
    in them.
    '''
    columns = ['PLAYER',
               'POS',
               'PROJECTION']
    data=pd.DataFrame(data=[info for player  in[getProjections(prefix+str(i)) for i in range(max_+1) if i%mod==0]  
                            for info in player],columns=columns)
    data['POS']=data['POS'].apply(lambda x: x.split('\xa0')[1]) #get position now
    return data
    
def getProjections(link):
    '''
    Using a link gets the rank of projections of ESPN Fantasy Football players.
    Grabs the html of the page and then parses it to create a dataframe of the 
    player and his projected points.
    
    THESE PROJECTIONS ARE CURRENTLY NON-PPR
    '''
    
    #sets up links usage with beautifulsoup
    response = requests.get(link)
    html = response.text.encode()
    soup = BeautifulSoup(html,'html.parser')
    
    #get rankings by finding all the rows in each table. Take away the first two rows cause that's header information
    table = soup.find('table',{'id':'playertable_0'})
    rankings = [[y.text for y in x.find_all('td')] for x in table.find_all('tr')][2:]

    projections = [[player[1].split(',')[0],
                    player[1], #can't explicitly get position here, even though you should be able to, no idea why
                    float(player[-1])] for player in rankings]
    return projections

In [93]:
data = createDataframe(ESPN_PP_PREFIX_LINK)
data

,PLAYER,POS,PROJECTION
0,Le'Veon Bell,RB,351.8
1,Todd Gurley II,RB,329.4
2,David Johnson,RB,325.4
3,Antonio Brown,WR,322.2
4,Tom Brady,QB,317.0
5,Aaron Rodgers,QB,316.2
6,Cam Newton,QB,302.9
7,Ezekiel Elliott,RB,302.8
8,Russell Wilson,QB,294.0
9,Julio Jones,WR,291.9


In [94]:
data.to_csv('projections2018.csv')